## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-03-16-07-48 +0000,bbc,Disney names Josh D'Amaro as new chief executive,https://www.bbc.com/news/articles/cy4qxdq2147o...
1,2026-02-03-16-06-52 +0000,nypost,Mike Johnson gloats over ‘family squabble’ bet...,https://nypost.com/2026/02/03/us-news/house-sp...
2,2026-02-03-16-06-42 +0000,nypost,Cracker Barrel urges staffers to eat at chain ...,https://nypost.com/2026/02/03/business/cracker...
3,2026-02-03-16-05-33 +0000,nyt,Disney Names Josh D’Amaro as Next C.E.O.,https://www.nytimes.com/2026/02/03/business/di...
4,2026-02-03-16-03-00 +0000,wsj,Trump Says He Seeks $1 Billion ‘in Damages’ Fr...,https://www.wsj.com/us-news/education/trump-sa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2519/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,46
142,epstein,25
114,ice,14
4,new,14
286,deal,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
258,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...,109
51,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...,104
334,2026-02-02-18-39-33 +0000,nypost,"Trump says Jeffrey Epstein, Michael Wolff ‘con...",https://nypost.com/2026/02/02/us-news/trump-sa...,97
360,2026-02-02-17-14-00 +0000,wsj,President Trump’s choice of Kevin Warsh as cha...,https://www.wsj.com/economy/central-banking/wh...,93
226,2026-02-03-00-37-12 +0000,nyt,"Trump Announces Initial Trade Deal With India,...",https://www.nytimes.com/2026/02/02/us/politics...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
258,109,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...
51,93,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
47,62,2026-02-03-14-46-52 +0000,bbc,Son of Norway's crown princess hears details o...,https://www.bbc.com/news/articles/ckgn8dnxjgwo...
166,50,2026-02-03-08-30-23 +0000,nypost,Savannah Guthrie’s sister enjoyed dinner with ...,https://nypost.com/2026/02/03/us-news/savannah...
73,47,2026-02-03-13-32-11 +0000,latimes,Disney names theme parks head Josh D'Amaro as ...,https://www.latimes.com/entertainment-arts/bus...
360,47,2026-02-02-17-14-00 +0000,wsj,President Trump’s choice of Kevin Warsh as cha...,https://www.wsj.com/economy/central-banking/wh...
250,46,2026-02-02-22-57-52 +0000,cbc,U.S. judge grants asylum to Chinese whistleblo...,https://www.cbc.ca/radio/asithappens/china-whi...
322,42,2026-02-02-19-19-00 +0000,wsj,U.S. Will Cut Tariffs on India to 18% in Trade...,https://www.wsj.com/politics/policy/u-s-will-c...
244,41,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...
303,37,2026-02-02-20-20-26 +0000,wapo,Rafah crossing between Gaza and Egypt reopens ...,https://www.washingtonpost.com/world/2026/02/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
